In [52]:
from sklearn.feature_extraction.text import CountVectorizer
import re
import numpy as np

# Original Hangman Game

In [7]:
import random

class HangmanGame:
    def __init__(self, word_list):
        self.word_list = word_list
        self.target_word = ""
        self.guesses_left = 6
        self.guessed_letters = set()
        self.word_in_progress = []

    def choose_word(self):
        self.target_word = random.choice(self.word_list)
        self.word_in_progress = ["_"] * len(self.target_word)

    def display_word(self):
        return " ".join(self.word_in_progress)

    def make_guess(self, letter):
        if letter in self.guessed_letters:
            return "You already guessed that letter!"

        self.guessed_letters.add(letter)

        if letter in self.target_word:
            for i in range(len(self.target_word)):
                if self.target_word[i] == letter:
                    self.word_in_progress[i] = letter
            if "_" not in self.word_in_progress:
                return "Congratulations! You guessed the word: {}".format(self.target_word)
        else:
            self.guesses_left -= 1
            if self.guesses_left == 0:
                return "Sorry, you ran out of guesses. The word was: {}".format(self.target_word)

        return self.display_word() + f"\nLives left: {self.guesses_left}"

# Example usage:
word_list = ["python", "hangman", "coding", "challenge", "game"]

custom_hangman = HangmanGame(word_list)
custom_hangman.choose_word()

print("Welcome to Hangman!")
print("Word to guess:", custom_hangman.display_word())

while custom_hangman.guesses_left > 0 and "_" in custom_hangman.word_in_progress:
    guess = input("Enter your guess: ").lower()
    result = custom_hangman.make_guess(guess)
    print(result)

print("Game over.")


Welcome to Hangman!
Word to guess: _ _ _ _ _ _ _ _ _
Enter your guess: c
c _ _ _ _ _ _ _ _
Lives left: 6
Enter your guess: h
c h _ _ _ _ _ _ _
Lives left: 6
Enter your guess: a
c h a _ _ _ _ _ _
Lives left: 6
Enter your guess: e
c h a _ _ e _ _ e
Lives left: 6
Enter your guess: l
c h a l l e _ _ e
Lives left: 6
Enter your guess: n
c h a l l e n _ e
Lives left: 6
Enter your guess: g
Congratulations! You guessed the word: challenge
Game over.


## Altering

In [51]:
import random

class HangmanGame:
    def __init__(self, word_list):
        self.word_list = word_list
        self.target_word = ""
        self.guesses_left = 6
        self.guessed_letters = set()
        self.word_in_progress = []
        self.common_letters = ''

    def choose_word(self):
        self.target_word = random.choice(self.word_list)
        self.word_in_progress = ["_"] * len(self.target_word)

    def display_word(self):
        return " ".join(self.word_in_progress)
    
    
    def guess_here(self):
        
        #Guessing the first word using random guess, based on MOST COMMON LETTERS
        random_guess_char = "eariotns"
        """
        Most Common Letters in percentage
        E – 11.1607%
        A – 8.4966%
        R – 7.5809%
        I – 7.5448%
        O – 7.1635%
        T – 6.9509%
        N – 6.6544%
        S – 5.7351%
        
        """
        if len(re.findall(r"[a-zA-Z]+","".join(self.word_in_progress)))== 0:
            for char in range(len(random_guess_char)):
                if random_guess_char[char] in self.common_letters:
                    continue
                else:
                    self.common_letters += random_guess_char[char]
                    return random_guess_char[char]

        #first take the incomplete word from word_in_progress
        incomplete_word = "".join(self.word_in_progress)
        #Find all the same length words from the final_list to predict the next letter
        words_with_same_length = [words for words in self.word_list if len(char) == len(self.word_in_progress)]
        
        # Convert them into vectors using CountVectorizer
        vectorizer = CountVectorizer(analyzer = 'char')
        incomplete_word_vector = vectorizer.transform(incomplete_word)
        words_with_same_length_vector = vectorizer.fit_transform(words_with_same_length)
        similarities = cosine_similarity(words_with_same_length_vector, incomplete_word_vector)
        
        

    def make_guess(self, letter):
        if letter in self.guessed_letters:
            return "You already guessed that letter!"

        self.guessed_letters.add(letter)

        if letter in self.target_word:
            for i in range(len(self.target_word)):
                if self.target_word[i] == letter:
                    self.word_in_progress[i] = letter
            if "_" not in self.word_in_progress:
                return "Congratulations! You guessed the word: {}".format(self.target_word)
        else:
            self.guesses_left -= 1
            if self.guesses_left == 0:
                return "Sorry, you ran out of guesses. The word was: {}".format(self.target_word)

        return self.display_word() + f"\nLives left: {self.guesses_left}"

# Example usage:
word_list = ["python", "hangman", "coding", "challenge", "game"]
#word_list = ['aion']
custom_hangman = HangmanGame(word_list)
custom_hangman.choose_word()

print("Welcome to Hangman!")
print("Word to guess:", custom_hangman.display_word())

while custom_hangman.guesses_left > 0 and "_" in custom_hangman.word_in_progress:
    #guess = input("Enter your guess: ").lower()
    guess = custom_hangman.guess_here()
    result = custom_hangman.make_guess(guess)
    print(result)

print("Game over.")


Welcome to Hangman!
Word to guess: _ _ _ _ _ _
_ _ _ _ _ _
Lives left: 5
_ _ _ _ _ _
Lives left: 4
_ _ _ _ _ _
Lives left: 3
_ _ _ i _ _
Lives left: 3


TypeError: 'in <string>' requires string as left operand, not NoneType

'absj'

In [41]:
word = 'hangmn'
word_in_progress = ["_"] * 6
word_in_progress

['_', '_', '_', '_', '_', '_']

In [ ]:
word_in_progress = ["_"] * 6
incomplete_word = "".join(word_in_progress)
#Find all the same length words from the final_list to predict the next letter
words_with_same_length = [words for words in self.word_list if len(char) == len(self.word_in_progress)]

# Convert them into vectors using CountVectorizer
vectorizer = CountVectorizer(analyzer = 'char')
incomplete_word_vector = vectorizer.transform(incomplete_word)
words_with_same_length_vector = vectorizer.fit_transform(words_with_same_length)
similarities = cosine_similarity(words_with_same_length_vector, incomplete_word_vector)